# Trasnformaciones con estado (*stateful transformations*)

### updateStateByKey

La operación `updateStateByKey` permite mantener el estado anterior de la información recibida por el sistema para actualizarla cuando llega nueva información. Para ello:

1. Se define el estado (cualquier tipo de dato)
2. Se define una función de actualización, que implementa cómo actualizar el estado utilizando el estado previo y los nuevos valores para cada clave definida en el stream de datos.

La función de actualización se aplica a todas las claves existentes, tanto si se reciben nuevos valores en el batch de datos como si no. Si la función devuelve None se elimina el par clave-valor.

Se requiere la configuración del directorio de checkpoint para utilizar la función `updateStateByKey`.

### Exercise

In [ ]:
import findspark
#Importante: Modificar la ruta para que apunte al HOME de Spark
findspark.init('/opt/spark')

In [ ]:
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
sc = SparkContext()
ssc = StreamingContext(sc, 1)

#Se define el directorio de checkpoint (necesario para utilizar la función updateStateByKey)
ssc.checkpoint('checkpoint_state')

### Introducción de datos
Se va a utilizar un socket para introducir datos. Dicho socket estará escuchando en el puerto 9999 de localhost. Para la introducción de datos se puede abrir una ventana de comandos y ejecutar el comando "nc -lk 9999", que abre el puerto y lo mantiene abierto, para a continuación pegar los mensajes que se quieren enviar. Si no existe el comando, deberemos instalar el programa "netcat" --> "sudo apt-get install netcat"

In [ ]:
ds = ssc.socketTextStream("localhost", 9999)

La función de actualización simplemente suma todos los nuevos valores de una clave en el nuevo batch de datos analizados (recibidas en *newValues*) y añade el valor total al valor previo (almacenado en *runningCount*).

In [ ]:
def updateFunction(newValues, runningCount):
    return sum(newValues) + (runningCount or 0)

El programa que se va a implementar recibe el DStream de datos numéricos, aplica la operación *mod 10* a cada valor recibido, y almacena cuántas veces ha aparecido cada número del 0 al 9. Finalmente también imprime el número total de claves recibidas.

In [ ]:
dst = ds.map(lambda x: int(x) % 10 if x.isdigit() else 0).map(lambda x: (x,1)).updateStateByKey(updateFunction)

dst.pprint()
dst.count().pprint()

In [ ]:
ssc.start()

In [ ]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)